# Running HDBSCAN on partial distance matrices?

What can we get from HDBSCAN on a sparse distance matrix having non-zero entries only between high dimensional k-NN data pairs? 
* First we cheat: the distance values are the low dimensional distances (after dimension reduction with UMAP). If we use a small number of NNs (same as for the UMAP projection say), running HDBSCAN on the sparse matrix does not provide good clusters. The larger the number of NNs - the larger the number of non-zero values - the closer we get to HDBSCAN result on UMAP reduction. I am still surprised that by *cheating* a lot and providing a kNN graph (with large k value) of the low dim distances, we do not get the same results as HDBSCAN on the low dim points. This brings us to the next natural question: how do NNs correspond between high and low dimensions?
* from UMAP weighted adjacency matrix with unknown values encoded as infinite values. - I actually don't know how to turn the UMAP weight into a reasonable distance. I just used the inverse. (doesn't work!)

In [1]:
!git branch

* main


In [66]:
execfile('functions/data_specifics.py')
execfile('functions/graph_functions.py')
print(data_set_list)

['pendigits', 'coil', 'mnist', 'usps', 'buildings']


In [16]:
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score, silhouette_score
from IPython.display import display, Markdown, Latex

import numpy as np
import pandas as pd
import requests

import matplotlib.pyplot as plt
import seaborn as sns

import hdbscan
import umap
from sklearn.neighbors import KNeighborsTransformer
import pynndescent

import networkx as nx
import itertools
import collections
import igraph as ig
from scipy.spatial.distance import euclidean

sns.set()

In [17]:
def enrich_graph_edge_properties(G, vertex_high_representation=None, vertex_low_representation=None):
    
    G.es['umap_weight'] = G.es['weight']
    
    simplices = G.cliques(min=3, max=3)
    edge_in_simplex = []
    for x in simplices:
        edge_in_simplex += G.get_eids([pair for pair in itertools.combinations(x, r=2)], directed=False)
    G.es["nb_triangles"] = 0
    for k, v in collections.Counter(edge_in_simplex).items():
        G.es[k]["nb_triangles"] = v
    
    if(vertex_low_representation is not None):
        G.es['lowdim_dist'] = [euclidean( vertex_low_representation[e.source], 
                                         vertex_low_representation[e.target] ) 
                               for e in G.es]
        
    if(vertex_high_representation is not None):
        G.es['highdist_dist'] = [euclidean( vertex_high_representation[e.source], 
                                         vertex_high_representation[e.target] ) 
                               for e in G.es] 
        
    for v in G.vs:
        x = {e:G.es[e]['highdist_dist'] for e in G.incident(v)}
        for i,e in enumerate(sorted(x, key=x.get)):
            G.es[e]['highdist_rank'] = i
            
    for v in G.vs:
        x = {e:G.es[e]['lowdim_dist'] for e in G.incident(v)}
        for i,e in enumerate(sorted(x, key=x.get)):
            G.es[e]['lowdist_rank'] = i
    
    for v in G.vs:
        x = {e:G.es[e]['umap_weight'] for e in G.incident(v)}
        for i,e in enumerate(sorted(x, key=x.get)):
            G.es[e]['umap_rank'] = i
    return(G)

In [6]:
def cluster_edge_pruning_triangle(G, min_triangle=0):
    G_pendigits_filter = G_pendigits.copy()
    edges_rm = [(u,v) for u,v,e in G_pendigits_filter.edges(data=True) if e['nb_triangles']<min_triangle]
    G_pendigits_filter.remove_edges_from(edges_rm)
    
    cc_labels = [-1]*G_pendigits.number_of_nodes()
    cc_iter = list(nx.connected_components(G_pendigits_filter))
    for i, c in enumerate(cc_iter):
        if(len(c)<2):
            continue
        for x in c:
            cc_labels[x] = i
    return(cc_labels)

In [59]:
def hdbscan_on_cc(A, min_samples, min_cluster_size):
    A_cc = scipy.sparse.csgraph.connected_components(A, return_labels=True)
    final_clusters = np.array([-1]*A.shape[0])
    hd_umap_labels_cc = dict()
    d0 = {-1:-1}
    for i in range(A_cc[0]):
        w = (A_cc[1]==i)
        n_points = sum(w)
        print(n_points)
        if(n_points>min_cluster_size):
            hd_umap_labels_cc[i] = hdbscan.HDBSCAN(min_samples=min_samples, min_cluster_size=min_cluster_size, metric='precomputed', allow_single_cluster=True, max_dist=2*A.max()).fit_predict(A[w, :][:, w])
            m = max(hd_umap_labels_cc[i])
            M = max(final_clusters)
            d1 = {i:M+i+1 for i in range(m+1)}
            new_cluster_id = {**d0, **d1} 
            print(new_cluster_id)
            final_clusters[w] = [new_cluster_id[c] for c in hd_umap_labels_cc[i]]
    return(final_clusters)

In [60]:
def hdbscan_connect(A, min_samples, min_cluster_size):
    A_cc = scipy.sparse.csgraph.connected_components(A, return_labels=True)
    first_occ = []
    m = max(A.data)
    for i in range(A_cc[0]):
        first_occ.append(np.where(A_cc[1] == i)[0][0])
        if(i>0):
            A[first_occ[i-1], first_occ[i]] = A[first_occ[i], first_occ[i-1]] = 2*m
    final_clusters = hdbscan.HDBSCAN(min_samples=min_samples, min_cluster_size=min_cluster_size, metric='precomputed', max_dist=2*A.max()).fit_predict(A)
    return(final_clusters)

In [9]:
knn_umap = 15
knn_graph = 40

# Pendigits clustering scores

In [47]:
dataset_id = 0
raw_data, targets, dataset_name = get_dataset(dataset_id)

G, A, dists = get_umap_graph(raw_data, dataset_id=dataset_id, set_op_mix_ratio=1.0, return_all=True)
umap_rep = get_umap_vectors(dataset_id=dataset_id, raw_data=raw_data)

In [48]:
G = enrich_graph_edge_properties(G, 
                             vertex_high_representation=raw_data, 
                             vertex_low_representation=umap_rep)

In [49]:
G.es[0]

igraph.Edge(<igraph.Graph object at 0x7fd176781750>, 0, {'weight': 0.16689156, 'umap_weight': 0.16689156, 'nb_triangles': 3, 'lowdim_dist': 0.28059548139572144, 'highdist_dist': 20.784608840942383, 'highdist_rank': 8, 'lowdist_rank': 7, 'umap_rank': 5})

In [50]:
D0 = A.copy()
for e in G.es:
    D0[e.target, e.source] = D0[e.source, e.target] = e['lowdim_dist']

In [51]:
params = get_dataset_params(dataset_id)
hd_umap_labels0 = hdbscan.HDBSCAN(min_samples=params['min_samples'], min_cluster_size=params['min_cluster_size'], metric='precomputed').fit_predict(D0)
hd_umap_labels = hdbscan.HDBSCAN(min_samples=params['min_samples'], min_cluster_size=params['min_cluster_size']).fit_predict(umap_rep)

In [52]:
print('Evaluation of clustering on lower dim data points')
print(f'{max(hd_umap_labels)+1} clusters')
print(f'ARI : {adjusted_rand_score(targets, hd_umap_labels)}')
print(f'AMI : {adjusted_mutual_info_score(targets, hd_umap_labels)}')
print('Evaluation of clustering on partial distance matrix')
print(f'{max(hd_umap_labels0)+1} clusters')
print(f'ARI : {adjusted_rand_score(targets, hd_umap_labels0)}')
print(f'AMI : {adjusted_mutual_info_score(targets, hd_umap_labels0)}')
print('Similarity between the two partitions')
print(f'ARI : {adjusted_rand_score(hd_umap_labels0, hd_umap_labels)}')
print(f'AMI : {adjusted_mutual_info_score(hd_umap_labels0, hd_umap_labels)}')

Evaluation of clustering on lower dim data points
10 clusters
ARI : 0.9185149200427103
AMI : 0.9320899303214291
Evaluation of clustering on partial distance matrix
10 clusters
ARI : 0.9163295428694507
AMI : 0.9293292186972308
Similarity between the two partitions
ARI : 0.9961603978065179
AMI : 0.9938411946343767


In [61]:
def hdbscan_on_partial_dist(dataset_id):
    raw_data, targets, dataset_name = get_dataset(dataset_id)
    display(Markdown(f'## {dataset_name}'))

    G, A, dists = get_umap_graph(raw_data, dataset_id=dataset_id, set_op_mix_ratio=1.0, return_all=True)
    umap_rep = get_umap_vectors(dataset_id=dataset_id, raw_data=raw_data)
    
    G = enrich_graph_edge_properties(G, 
                             vertex_high_representation=raw_data, 
                             vertex_low_representation=umap_rep)
    
    D0 = A.copy()
    for e in G.es:
        D0[e.target, e.source] = D0[e.source, e.target] = e['lowdim_dist']
    D_cc = scipy.sparse.csgraph.connected_components(D0, return_labels=True)
        
    params = get_dataset_params(dataset_id)
    if(D_cc[0]==1):
        hd_umap_labels0 = hdbscan.HDBSCAN(min_samples=params['min_samples'], min_cluster_size=params['min_cluster_size'], metric='precomputed', max_dist=2*D0.max()).fit_predict(D0)
    else:
        hd_umap_labels0 = hdbscan_connect(D0, min_samples=params['min_samples'], min_cluster_size=params['min_cluster_size'])
    hd_umap_labels = hdbscan.HDBSCAN(min_samples=params['min_samples'], min_cluster_size=params['min_cluster_size']).fit_predict(umap_rep)  
    
    print('Evaluation of clustering on lower dim data points')
    print(f'{max(hd_umap_labels)+1} clusters')
    print(f'ARI : {adjusted_rand_score(targets, hd_umap_labels)}')
    print(f'AMI : {adjusted_mutual_info_score(targets, hd_umap_labels)}')
    print('Evaluation of clustering on partial distance matrix')
    print(f'{max(hd_umap_labels0)+1} clusters')
    print(f'ARI : {adjusted_rand_score(targets, hd_umap_labels0)}')
    print(f'AMI : {adjusted_mutual_info_score(targets, hd_umap_labels0)}')
    print('Similarity between the two partitions')
    print(f'ARI : {adjusted_rand_score(hd_umap_labels0, hd_umap_labels)}')
    print(f'AMI : {adjusted_mutual_info_score(hd_umap_labels0, hd_umap_labels)}')

In [63]:
# For each data set, run the above analysis
for i in range(5):
    hdbscan_on_partial_dist(dataset_id=i)

## pendigits

Evaluation of clustering on lower dim data points
10 clusters
ARI : 0.9185149200427103
AMI : 0.9320899303214291
Evaluation of clustering on partial distance matrix
10 clusters
ARI : 0.9163295428694507
AMI : 0.9293292186972308
Similarity between the two partitions
ARI : 0.9961603978065179
AMI : 0.9938411946343767


## coil

/disk/home/vmpouli/.conda/envs/high-dim-easydata/lib/python3.7/site-packages/sklearn/manifold/_spectral_embedding.py:261: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  "Graph is not fully connected, spectral embedding may not work as expected."
/disk/home/vmpouli/.conda/envs/high-dim-easydata/lib/python3.7/site-packages/scipy/sparse/_index.py:82: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


Evaluation of clustering on lower dim data points
19 clusters
ARI : 0.7967647891406653
AMI : 0.9462144192674433
Evaluation of clustering on partial distance matrix
19 clusters
ARI : 0.7962790399627984
AMI : 0.9454387450892846
Similarity between the two partitions
ARI : 0.980754849586676
AMI : 0.9875228346539583


...

## usps

Evaluation of clustering on lower dim data points
9 clusters
ARI : 0.8824676944734986
AMI : 0.9004545663772475
Evaluation of clustering on partial distance matrix
9 clusters
ARI : 0.6128864462599586
AMI : 0.7564780387572362
Similarity between the two partitions
ARI : 0.6549619081041632
AMI : 0.7870445119824904


## buildings

/disk/home/vmpouli/.conda/envs/high-dim-easydata/lib/python3.7/site-packages/scipy/sparse/_index.py:82: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


Evaluation of clustering on lower dim data points
100 clusters
ARI : 0.2458851298996959
AMI : 0.6307848503576966
Evaluation of clustering on partial distance matrix
92 clusters
ARI : 0.3013433418091169
AMI : 0.638557956800602
Similarity between the two partitions
ARI : 0.6905099857802873
AMI : 0.9246471394105523
